In [23]:
import pandas as pd

# plotly viz
import plotly.graph_objects as go
import plotly.express as px

# cleanair modules
from cleanair.databases import DBReader



In [31]:
class ScootQuery(DBReader):
    """
    Queries to run on the SCOOT DB.
    """
    
    def groupby_datetime_df(self, start_datetime="2020-02-23"):
        """
        Group by hour and return the average or sum of the column for that hour.
        """
        query = """
            SELECT measurement_start_utc, measurement_end_utc,
                SUM(n_vehicles_in_interval) AS sum_n_vehicles_in_interval,
                AVG(occupancy_percentage) as avg_occupancy_percentage,
                AVG(congestion_percentage) as avg_congestion_percentage,
                AVG(saturation_percentage) as avg_saturation_percentage,
                SUM(flow_raw_count) as sum_flow_raw_count,
                SUM(occupancy_raw_count) as sum_occupancy_raw_count,
                SUM(congestion_raw_count) as sum_congestion_raw_count,
                SUM(saturation_raw_count) as sum_saturation_raw_count
            FROM dynamic_data.scoot_reading
            where measurement_start_utc > '{start}'
            group by measurement_start_utc, measurement_end_utc
            order by measurement_start_utc;
        """.format(start=start_datetime)
        print(query)
    
        with self.dbcnxn.open_session() as session:
            df = pd.read_sql(query, session.bind)
            return df
        
    def get_all_readings(self, start_datetime="2020-02-23"):
        """
        Get every reading for every SCOOT detector + the lat and lon of the sensor.
        """
        query = """
            SELECT detector_id, ST_X(interest_points.meta_point."location") as "lon",
                ST_Y(interest_points.meta_point."location") as "lat",
                measurement_start_utc, measurement_end_utc,
                n_vehicles_in_interval, occupancy_percentage,
                congestion_percentage, saturation_percentage as "saturation"
            FROM dynamic_data.scoot_reading 
            JOIN interest_points.scoot_detector on detector_id = interest_points.scoot_detector.detector_n 
            JOIN interest_points.meta_point on id = interest_points.scoot_detector.point_id
            WHERE measurement_start_utc > '{start}';
        """.format(start=start_datetime)
        
        with self.dbcnxn.open_session() as session:
            df = pd.read_sql(query, session.bind)
            return df

In [32]:
secretfile = "../../terraform/.secrets/db_secrets.json"

SQ = ScootQuery(secretfile=secretfile)

2020-03-23 17:59:00     INFO: Database connection information loaded from <_io.TextIOWrapper name='../../terraform/.secrets/db_secrets.json' mode='r' encoding='UTF-8'>


In [33]:
df = SQ.groupby_datetime_df()


            SELECT measurement_start_utc, measurement_end_utc,
                SUM(n_vehicles_in_interval) AS sum_n_vehicles_in_interval,
                AVG(occupancy_percentage) as avg_occupancy_percentage,
                AVG(congestion_percentage) as avg_congestion_percentage,
                AVG(saturation_percentage) as avg_saturation_percentage,
                SUM(flow_raw_count), SUM(occupancy_raw_count), SUM(congestion_raw_count), SUM(saturation_raw_count)
            FROM dynamic_data.scoot_reading
            where measurement_start_utc > '2020-02-23'
            group by measurement_start_utc, measurement_end_utc
            order by measurement_start_utc;
        


In [34]:
df.sample(10)

,measurement_start_utc,measurement_end_utc,sum_n_vehicles_in_interval,avg_occupancy_percentage,avg_congestion_percentage,avg_saturation_percentage,sum,sum,sum,sum
175,2020-03-01 08:00:00,2020-03-01 09:00:00,2231076,5.835409,0.956631,21.029728,365648002,338475183,318345143,272720672
327,2020-03-07 16:00:00,2020-03-07 17:00:00,4364523,15.756959,4.082267,53.579920,368597511,338498324,318508058,314471882
96,2020-02-27 01:00:00,2020-02-27 02:00:00,1128110,3.188822,0.609862,12.197767,366015657,335984829,318655192,246720523
54,2020-02-25 07:00:00,2020-02-25 08:00:00,4340279,16.818852,4.821312,53.233809,374091083,347587035,327754866,315098503
272,2020-03-05 09:00:00,2020-03-05 10:00:00,4247391,18.367145,5.671441,58.019387,374233063,345107744,326179315,325595245
141,2020-02-28 22:00:00,2020-02-28 23:00:00,3217523,9.458879,1.835100,34.754401,372189614,343020379,325330898,295492730
51,2020-02-25 04:00:00,2020-02-25 05:00:00,906896,2.508884,0.441621,9.594179,364369076,334977825,317288722,233990946
159,2020-02-29 16:00:00,2020-02-29 17:00:00,4400434,17.141500,5.005683,56.114578,370146414,339937908,319868892,315755550
90,2020-02-26 19:00:00,2020-02-26 20:00:00,4242361,14.792143,3.687107,51.774688,371263592,342312425,322872758,317926269
390,2020-03-10 07:00:00,2020-03-10 08:00:00,4329537,16.643606,4.646068,52.521533,375094955,348381931,328390770,315814912


In [36]:
col = "sum_n_vehicles_in_interval"

all_sensors_fig = dict(
    data=[
        dict(
            x=df.measurement_start_utc,
            y=df[col],
            mode="lines",
        )
    ],
    layout=dict(
        title="{col} over time.".format(col=col),
    )
)

go.Figure(all_sensors_fig)

In [37]:
# get all data for the timeperiod
start_datetime = "2020-03-11"
all_scoot_df = SQ.get_all_readings(start_datetime=start_datetime)

In [45]:
import os

# set the mapbox token
secrets_dir = "../../terraform/.secrets"
mapbox_filepath = os.path.join(secrets_dir, ".mapbox_token")
mapbox_access_token = open(mapbox_filepath).read()
px.set_mapbox_access_token(mapbox_access_token)

col = "n_vehicles_in_interval"
lower_bound = 500
timestamp = "2020-03-12 12:00:00"

# DECREASE NUMBER OF ROWS IN DATAFRAME
# otherwise we will have problems rendering cause too many scoot sensors
filtered_df = all_scoot_df.loc[
    (all_scoot_df[col] > lower_bound) & (all_scoot_df["measurement_start_utc"] == timestamp)
]

# get the mapbox figure
map_dict = px.scatter_mapbox(
    filtered_df,
    lat="lat",
    lon="lon",
    size=[2 for i in range(len(filtered_df.index))],
    color=col,
    zoom=10,
    mapbox_style="basic",
)
map_fig = go.FigureWidget(map_dict)
map_fig.show()